In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

from utils import *
import json
import random
#from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish

from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import re
import string
from sklearn.metrics import classification_report, accuracy_score, make_scorer


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *

# DEEP LEARNING IMPORTS
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Activation, Dropout, Flatten, MaxPooling2D,Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix


from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from sklearn.pipeline import Pipeline


In [11]:
#column = ['tweets','duygu','preprocessing']
#df = pd.read_excel("../dataset/total.xlsx")

column = ['tweets','duygu']
df = pd.read_excel("../dataset/kemik_pos_neg.xlsx")


df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [12]:
df.loc[df.duygu=="olumlu","duygu"]=1
df.loc[df.duygu=="olumsuz","duygu"]=0
df[:4]

,tweets,duygu
0,turkcell heryerde çekiyor kesin bilgi,1
1,turkcell olmak ayrıcalıktir çünkü kuzenlerin v...,1
2,allahtan turkcell'liyim amin,1
3,avea kaşar yaşasın turkcell,1


In [13]:
df=df.sample(frac=1).reset_index(drop=True)
df

,tweets,duygu
0,2 saatir reklam veriyorsun ama hala çekmiyorsu...,0
1,turkcell grubu son 10 yilin geli̇r ve büyüme r...,1
2,turkcell görme engelliler icin yaptiginiz rekl...,1
3,turkcell reklamındaki gibi bir sarkiyla kiz ta...,0
4,hayret turkcell bana beleş 2500 sms verdi nolu...,1
...,...,...
9121,new post: turkcell fatura itiraz dilekçesi örn...,0
9122,turkcell sayesinde sabretmeyi öğrendim,1
9123,turkcell 14 gb internet verdi ama sadece bir g...,0
9124,kalktık ama halen internetteyim yaşa turkcell ...,1


In [14]:
X_t=df['tweets'].tolist()

In [15]:
# Creating a tokenizer
tokenizer = Tokenizer(lower=True)
# Building word indices
tokenizer.fit_on_texts(X_t)
# Tokenizing sentences
sentences = tokenizer.texts_to_sequences(X_t)
# Creating a reverse dictionary
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
# Function takes a tokenized sentence and returns the words
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)
# Creating texts 
X_t = list(map(sequence_to_text, sentences))


In [16]:
from gensim.models import Word2Vec
import time
start=time.time()
n_features=32
window_size=5
min_count=1
epoch=50
n_workers=8

wordembeddings = Word2Vec(size = n_features,
            window = window_size, 
            min_count= min_count,
            workers = n_workers, 
            sg=1)
wordembeddings.build_vocab(X_t)
wordembeddings.train(X_t, 
            total_examples=wordembeddings.corpus_count,  
            epochs = epoch)
end=time.time()
print(end-start)

14.96999740600586


In [17]:
inputs=df['tweets'].to_numpy()
targets=df['duygu'].to_numpy()

In [18]:
def word2vec (X_train,y_train,X_test,y_test):
    #Create a tokenizer, configured to only take into account the 20 most common words çok küçük olursa kelimeleri 
    #kaybederiz underfit yaparız
    #tokenizer = Tokenizer(lower=True) #en yaygın kaç kelimeyi dikkate alacağı. Belirtilecek en iyi kelime sayısı #1000 yapan da var
    tokenizer.fit_on_texts(X_train) #keras tokenizer ile metni dictionary haline getiriyor.
    sequences_X_train = tokenizer.texts_to_sequences(X_train) #kelimelerin dictionarydeki karşılığı 
    #[[2, 1, 3], [2, 1], [4, 1], [5, 6]] şekline getiriliyor. 2-machine 1- learning 3-Knowledge 
    word_index = tokenizer.word_index #dictionarydeki kelimelerin sayısal karşılığı 'unk': 1, 'ürün': 2,
    max_length = 0
    for review_number in range(len(sequences_X_train)): #len(sequences_X_train) ile kaç tane [[2,3,4],[2,6]] var bulunuyor burda 2
        numberofwords=len(sequences_X_train[review_number]) #[2,3,4] içinde kaç tane şey var 3 burda
        if (numberofwords) > (max_length):
            max_length = numberofwords #tüm kelimelere bakıp en uzun olanı buluyor

    X_train = pad_sequences(sequences_X_train, maxlen=max_length) #ikili boyutlu matrise çevirip her cümelnin uzunluğunu eşit yapıyor.
    #En uzun cümle uzunluğuna tamamlanıyor.[[2 1 3] [0 2 1]] alt alta gelecek şekilde en uzun 6 ise 6x6 matris oluyor
    y_train = np.asarray(y_train) #tek boyutlu bir matris oluyor [1 1 0 ... 0 1 0] gibi

    sequences_X_test = tokenizer.texts_to_sequences(X_test) #train için yapılan gibi dictionary alınıyor
    X_test = pad_sequences(sequences_X_test, maxlen=max_length) #en uzun olana göre pad sequence yapılıyor
    y_test = np.asarray(y_test)


    unique_words = len(word_index) #word_index ile unique olan kelimeler alınıyor 0 dan başladığı için bir arttırılıyor
    total_words = unique_words + 1
    skipped_words = 0
    embedding_dim = 32 #embedding dim vector size ile aynı 
    embedding_vector=0
    embedding_matrix = np.zeros((total_words, embedding_dim))
    for word, index in tokenizer.word_index.items(): #kelime ve kelimenin dictionarydeki karşılığı alınıyor
        try:
            embedding_vector = wordembeddings[word]#wordembeddings.word_vectors[wordembeddings.dictionary[word]] #kelimenin word2vec karşılığı vektör olarak

        except:
            skipped_words = skipped_words+1
            pass
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector #dictionarydeki indexine word2vec teki sayısal hali yazılır
            
    embedding_layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)
    
    return embedding_layer,X_train,y_train,X_test,y_test

In [19]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]
pat=5
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=1)

for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= word2vec (X_train,y_train,X_test,y_test)
    
    model = Sequential() #rnn
    model.add(embedding_layer)
    model.add(SimpleRNN(128,activation='relu',return_sequences= True))
    model.add(SimpleRNN(256,activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    model.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = model.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

<ipython-input-18-7d0886ed6bc1>:32: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  embedding_vector = wordembeddings[word]#wordembeddings.word_vectors[wordembeddings.dictionary[word]] #kelimenin word2vec karşılığı vektör olarak


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/50
257/257 [==============================] - 12s 46ms/step - loss: 0.5213 - acc: 0.7398 - precision_1: 0.7480 - recall_1: 0.7279
Epoch 2/50
257/257 [==============================] - 11s 45ms/step - loss: 0.4610 - acc: 0.7812 - precision_1: 0.7928 - recall_1: 0.7649
Epoch 3/50
257/257 [==============================] - 14s 54ms/step - loss: 0.4272 - acc: 0.8012 - precision_1: 0.8162 - recall_1: 0.7804
Epoch 4/50
257/257 [==============================] - 13s 49ms/step - loss: 0.4000 - acc: 0.8237 - precision_1: 0.8360 - recall_1: 0.8080
Epoch 5/50
257/257 [==============================] - 16s 61ms/step - loss: 0.3568 - acc: 0.8406 - precision_1: 0.8526 - recall_1: 0.82600s - loss: 0.3563 - acc: 0.8410 - precision_1: 0.8526 - recall_1
Epoch 6/50
257/257 [==============================] - 22s 86ms/step - loss: 0.3244 - acc: 0.8589 - precision_1: 0.8664 - recall_1: 0.8506: 13s - loss

257/257 [==============================] - 24s 93ms/step - loss: 0.3944 - acc: 0.8230 - precision_3: 0.8376 - recall_3: 0.8027
Epoch 5/50
257/257 [==============================] - 24s 93ms/step - loss: 0.2188 - acc: 0.9112 - precision_3: 0.9144 - recall_3: 0.9080: 19s - loss: 0 - ETA: 1s - loss: 0.2196 - acc: 0.9107 - precision_3: 0.9148 - recall - ETA: 1s - loss: 0.2195 - acc: 0.9105 - precis
Epoch 9/50
257/257 [==============================] - 24s 94ms/step - loss: 0.1795 - acc: 0.9312 - precision_3: 0.9366 - recall_3: 0.92554s - loss: 0.1792 - acc: 0.9296 -  - ETA: 2s - loss: 0.1
Epoch 10/50
257/257 [==============================] - 19s 74ms/step - loss: 0.1291 - acc: 0.9517 - precision_3: 0.9502 - recall_3: 0.9536
Epoch 11/50
257/257 [==============================] - 19s 74ms/step - loss: 0.1204 - acc: 0.9543 - precision_3: 0.9559 - recall_3: 0.9529
Epoch 12/50
257/257 [==============================] - 18s 71ms/step - loss: 0.0980 - acc: 0.9659 - precision_3: 0.9656 - recall_3

257/257 [==============================] - 24s 92ms/step - loss: 0.0943 - acc: 0.9659 - precision_4: 0.9686 - recall_4: 0.9630: 18s - loss: 0 - ETA: 17s - loss: 0.0
Epoch 13/50
257/257 [==============================] - 24s 92ms/step - loss: 0.0866 - acc: 0.9709 - precision_4: 0.9724 - recall_4: 0.9693: 21s - loss: 0.0616 - acc: 0.9661 - precision_4: 0.9534 - recall_ - ETA: 20s - loss: 0.0560 - acc: 0.9762 - precision_4:  - ETA: 19s - loss: 0.0577 - acc: 0.9803  - ETA: 5s - loss: 0.0683 - acc: 0.9773 - precision_4: 0.9 - E - ETA: 0s - loss: 0.0852 - acc: 0.9718 - precision_4: 0.9745 -  - ETA: 0s - loss: 0.0863 - acc: 0.9712 - precision_4: 0.9731 - recall_4: 
Epoch 14/50
257/257 [==============================] - 23s 90ms/step - loss: 0.0831 - acc: 0.9691 - precision_4: 0.9695 - recall_4: 0.9686: 19s - loss: 0.0639 - acc: 0.9757 -
Epoch 15/50
257/257 [==============================] - 23s 91ms/step - loss: 0.0604 - acc: 0.9826 - precision_4: 0.9846 - recall_4: 0.9805
Epoch 16/50
257/257

257/257 [==============================] - 24s 94ms/step - loss: 0.0185 - acc: 0.9953 - precision_5: 0.9956 - recall_5: 0.9949: 21s - loss: 0.0 - ETA: 2s - loss: 0.0170 - 
Epoch 35/50
257/257 [==============================] - 24s 93ms/step - loss: 0.0426 - acc: 0.9851 - precision_5: 0.9861 - recall_5: 0.98415s - loss: 0.0462 - acc: 0.9835 - precision_5: 0.9844 - recall_5: 0.98 - ETA: 5s - loss: 0.0462 - acc:
Epoch 00035: early stopping
29/29 [==============================] - 1s 33ms/step - loss: 1.3242 - acc: 0.7853 - precision_5: 0.7892 - recall_5: 0.7892
------------------------------------------------------------------------
Training for fold 6 ...
Epoch 1/50
257/257 [==============================] - 19s 73ms/step - loss: 0.5262 - acc: 0.7413 - precision_6: 0.7481 - recall_6: 0.7263: 16s - loss: 0.5719 - acc: 0.7105 - precision_6: 0.7146 - r - ETA: 14s - ETA: 1s - loss: 0.5315 - ac
Epoch 2/50
257/257 [==============================] - 16s 61ms/step - loss: 0.4634 - acc: 0.7836 - 

257/257 [==============================] - 23s 91ms/step - loss: 0.3043 - acc: 0.8738 - precision_7: 0.8731 - recall_7: 0.8750: 15s - loss: 0.2772 - acc: 0.8902 - precision_7: 0.8852 - recall_7: 0 - ETA: 15s - loss: 0.2817 - acc: 0.8882 - precision_7:  - ETA: 13s - loss: 0.2876 - acc: 0.8847 - precision_7: 0. - ETA: 11s - loss: 0.2924 - acc: 0.8816 - precision_7: 0.8797 - recall_7: 0.8 - ETA: 10s - loss: 0.2927 - acc: 0.8810 - precision_7: 0.8781 - ETA: 9s - loss: 0.2986 - acc: 0.8777 -  - ETA: 7s - loss: 0.3045 - acc: 0.8743 - precision_ - ETA: 6s -
Epoch 7/50
257/257 [==============================] - 24s 92ms/step - loss: 0.2578 - acc: 0.8930 - precision_7: 0.8951 - recall_7: 0.8905
Epoch 8/50
257/257 [==============================] - 24s 92ms/step - loss: 0.2011 - acc: 0.9216 - precision_7: 0.9256 - recall_7: 0.9171: 19s - loss: 0.1637 - acc: 0.9439 - precision_7: 0.9455 - recall_7:  - ETA: 18s - loss: 0.1616 - acc: 0.9438 - precision_7: 0.9453 - re - ETA: 17s - loss:
Epoch 9/50
2

257/257 [==============================] - 18s 69ms/step - loss: 0.0391 - acc: 0.9875 - precision_8: 0.9897 - recall_8: 0.9851
Epoch 21/50
257/257 [==============================] - 16s 62ms/step - loss: 0.0613 - acc: 0.9771 - precision_8: 0.9798 - recall_8: 0.9741
Epoch 22/50
257/257 [==============================] - 14s 55ms/step - loss: 0.0258 - acc: 0.9925 - precision_8: 0.9927 - recall_8: 0.9922
Epoch 23/50
257/257 [==============================] - 18s 70ms/step - loss: 0.0535 - acc: 0.9817 - precision_8: 0.9833 - recall_8: 0.9800
Epoch 24/50
257/257 [==============================] - 18s 71ms/step - loss: 0.0386 - acc: 0.9866 - precision_8: 0.9870 - recall_8: 0.9861
Epoch 25/50
257/257 [==============================] - 18s 71ms/step - loss: 0.0308 - acc: 0.9905 - precision_8: 0.9898 - recall_8: 0.9912
Epoch 26/50
257/257 [==============================] - 18s 71ms/step - loss: 0.0393 - acc: 0.9866 - precision_8: 0.9866 - recall_8: 0.9866
Epoch 27/50
257/257 [==================

257/257 [==============================] - 9s 33ms/step - loss: 0.0410 - acc: 0.9842 - precision_10: 0.9854 - recall_10: 0.9830
Epoch 00022: early stopping
29/29 [==============================] - 0s 13ms/step - loss: 1.2613 - acc: 0.7654 - precision_10: 0.7386 - recall_10: 0.8018


In [20]:
def Average(lst):
    return sum(lst) / len(lst)

In [21]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

test Accuracy: 0.764190
test precision: 0.768341
test recall: 0.759158
test f1_score: 0.762515
